In [57]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import tabula

## Importing Data

### Bus Ridership and Stop Usage Data

sources:
- https://data.wprdc.org/dataset/prt-transit-stop-usage
- https://data.wprdc.org/dataset/prt-monthly-average-ridership-by-route

In [60]:
ridership_df = pd.read_csv("Data/avg_ridership.csv")
ridership_df.head()

,_id,route,ridership_route_code,route_full_name,current_garage,mode,month_start,year_month,day_type,avg_riders,day_count
0,1,1,001,1 - FREEPORT ROAD,Ross,Bus,2017-01-01,201701,SAT.,969.50,4
1,2,4,004,4 - TROY HILL,Ross,Bus,2017-01-01,201701,SAT.,218.25,4
2,3,6,006,6 - SPRING HILL,Ross,Bus,2017-01-01,201701,SAT.,495.50,4
3,4,8,008,8 - PERRYSVILLE,Ross,Bus,2017-01-01,201701,SAT.,1480.00,4
4,5,11,011,11 - FINEVIEW,Ross,Bus,2017-01-01,201701,SAT.,208.00,4


In [61]:
stopuse_df = pd.read_csv("Data/wprdc_stop_data.csv")
stopuse_df.head()

,clever_id,stop_id,stop_name,direction,routes_ser,latitude,longitude,mode,shelter,stop_type,datekey,time_period,route_name,serviceday,total_ons,total_offs,days,avg_ons,avg_offs
0,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Sat,12.0,0.0,4,3.000000,0.00
1,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Sun,14.0,0.0,6,2.333333,0.00
2,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,69,Weekday,64.0,1.0,20,3.200000,0.05
3,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,201909,Pre-pandemic,P69,Weekday,39.0,0.0,20,1.950000,0.00
4,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,202001,Pre-pandemic,69,Sat,11.0,0.0,4,2.750000,0.00


### Weather Data

In [63]:
## Monthly average temperatures in Pittsburgh, PA
temp_url = "https://www.weather.gov/media/pbz/records/histemp.pdf"
temp_df = tabula.read_pdf(temp_url, pages = 'all')[2]

In [64]:
cols = ['Year', 'January', 'February', 'March', 
        'April', 'May', 'June', 'July', 'August', 
        'September', 'October', 'November', 'December', 'Annual']
temp_df.columns = cols

In [65]:
temps_2024 = [2024, 33.0, 39.5, 46.6, 56.9, 66.5, 72.8, 76.5, 73.8, 69.4, 57.0, 47.6, 36.6, 56.4]
temps_2025 = [2025, 24.1, 31.7, 47.4] + [np.nan for i in range(len(cols)-3)]

temps_2024_df = dict(zip(cols, temps_2024))
temps_2024_df = pd.DataFrame(temps_2024_df, index = [1])

temps_2025_df = dict(zip(cols, temps_2025))
temps_2025_df = pd.DataFrame(temps_2025_df, index = [1])

In [66]:
temp_df = pd.concat([temp_df, temps_2024_df, temps_2025_df])
temp_df.reset_index(drop = True, inplace = True)
temp_df.head()

,Year,January,February,March,April,May,June,July,August,September,October,November,December,Annual
0,1975,32.6,32.1,36.3,44.3,63.0,57.8,72.8,73.0,58.8,53.3,46.3,32.9,51.1
1,1976,23.5,37.2,45.2,50.6,55.6,68.4,67.4,65.3,59.9,45.9,33.1,23.9,48.0
2,1977,11.4,26.9,43.7,50.8,63.0,63.8,71.8,68.1,64.7,50.5,45.6,31.1,49.3
3,1978,22.6,20.9,36.9,51.0,60.2,69.4,73.0,71.4,66.2,49.1,43.0,32.7,49.7
4,1979,21.4,18.0,43.1,49.7,59.1,67.7,70.3,69.6,63.4,50.9,44.7,34.6,49.4


In [67]:
temp_df = temp_df.melt(id_vars = ['Year'], 
                       var_name = 'Month', 
                       value_name = 'avg_temp')
temp_df.head()

,Year,Month,avg_temp
0,1975,January,32.6
1,1976,January,23.5
2,1977,January,11.4
3,1978,January,22.6
4,1979,January,21.4


In [68]:
temp_df = temp_df[temp_df['Month'] != 'Annual'].copy()
temp_df = temp_df[temp_df['Year'] != '30 yr'].copy()

d = dict((v,k) for k,v in zip(range(1, 13), temp_df.Month.unique()))
temp_df['month_index'] = temp_df['Month'].map(d)
temp_df['year_month'] = temp_df['Year'].astype(str) + temp_df['month_index'].astype(str).str.zfill(2)

temp_df

,Year,Month,avg_temp,month_index,year_month
0,1975,January,32.6,1,197501
1,1976,January,23.5,1,197601
2,1977,January,11.4,1,197701
3,1978,January,22.6,1,197801
4,1979,January,21.4,1,197901
...,...,...,...,...,...
607,2021,December,40.9,12,202112
608,2022,December,33.0,12,202212
609,2023,December,42.7,12,202312
610,2024,December,36.6,12,202412


In [69]:
## Monthly total precipitation (inches) in Pittsburgh, PA
precip_url = "https://www.weather.gov/media/pbz/records/hisprec.pdf"
precip_df = tabula.read_pdf(precip_url, pages = 'all')[2]

In [70]:
precip_df.columns = ['Year', 'January', 'February', 'March', 
                     'April', 'May', 'June', 'July', 'August', 
                     'September', 'October', 'November', 'December', 'Annual']
precip_df.head()

,Year,January,February,March,April,May,June,July,August,September,October,November,December,Annual
0,1964,2.55,1.73,4.96,7.61,1.77,3.84,4.48,1.79,0.74,1.42,2.74,4.26,37.89
1,1965,3.84,2.98,3.16,1.79,1.21,2.31,1.82,3.26,4.07,2.82,2.35,0.63,30.24
2,1966,4.52,3.23,1.88,3.73,2.76,1.72,2.70,5.13,1.92,1.38,3.39,1.70,34.06
3,1967,1.06,2.54,6.10,4.41,5.21,0.90,4.54,2.67,1.61,2.05,3.07,2.22,36.38
4,1968,2.83,0.79,4.53,2.33,6.36,2.38,2.36,3.97,3.08,2.13,2.07,3.24,36.07


In [71]:
precip_df = precip_df.melt(id_vars = ['Year'], 
                           var_name = 'Month', 
                           value_name = 'total_precip')
precip_df.head()

,Year,Month,total_precip
0,1964,January,2.55
1,1965,January,3.84
2,1966,January,4.52
3,1967,January,1.06
4,1968,January,2.83


In [72]:
precip_df = precip_df[precip_df['Month'] != 'Annual'].copy()
precip_df = precip_df[precip_df['Year'] != '30 yr'].copy()

d = dict((v,k) for k,v in zip(range(1, 13), precip_df.Month.unique()))
precip_df['month_index'] = precip_df['Month'].map(d)
precip_df['year_month'] = precip_df['Year'].astype(str) + precip_df['month_index'].astype(str).str.zfill(2)

precip_df

,Year,Month,total_precip,month_index,year_month
0,1964,January,2.55,1,196401
1,1965,January,3.84,1,196501
2,1966,January,4.52,1,196601
3,1967,January,1.06,1,196701
4,1968,January,2.83,1,196801
...,...,...,...,...,...
750,2021,December,3.45,12,202112
751,2022,December,1.76,12,202212
752,2023,December,2.80,12,202312
753,2024,December,2.71,12,202412


### ACS Population Data

In [74]:
## TODO

## Merge Data

### Add weather data to bus data

In [77]:
stopuse_df['datekey'] = stopuse_df['datekey'].astype(str)
stopuse_df = stopuse_df.merge(precip_df[['year_month', 'total_precip']], how = 'left', left_on = 'datekey', right_on = 'year_month')
stopuse_df = stopuse_df.merge(temp_df[['year_month', 'avg_temp']], how = 'left', left_on = 'datekey', right_on = 'year_month')
stopuse_df.drop(columns = ['year_month_x', 'year_month_y'], inplace = True)
stopuse_df.head()

,clever_id,stop_id,stop_name,direction,routes_ser,latitude,longitude,mode,shelter,stop_type,...,time_period,route_name,serviceday,total_ons,total_offs,days,avg_ons,avg_offs,total_precip,avg_temp
0,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Sat,12.0,0.0,4,3.000000,0.00,5.34,69.0
1,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Sun,14.0,0.0,6,2.333333,0.00,5.34,69.0
2,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Weekday,64.0,1.0,20,3.200000,0.05,5.34,69.0
3,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,P69,Weekday,39.0,0.0,20,1.950000,0.00,5.34,69.0
4,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.76,Bus,No Shelter,Bus Stop,...,Pre-pandemic,69,Sat,11.0,0.0,4,2.750000,0.00,3.69,35.2


In [78]:
ridership_df['year_month'] = ridership_df['year_month'].astype(str)
ridership_df = ridership_df.merge(precip_df[['year_month', 'total_precip']], how = 'left', on = 'year_month')
ridership_df = ridership_df.merge(temp_df[['year_month', 'avg_temp']], how = 'left', on = 'year_month')
ridership_df.head()

,_id,route,ridership_route_code,route_full_name,current_garage,mode,month_start,year_month,day_type,avg_riders,day_count,total_precip,avg_temp
0,1,1,001,1 - FREEPORT ROAD,Ross,Bus,2017-01-01,201701,SAT.,969.50,4,3.54,34.6
1,2,4,004,4 - TROY HILL,Ross,Bus,2017-01-01,201701,SAT.,218.25,4,3.54,34.6
2,3,6,006,6 - SPRING HILL,Ross,Bus,2017-01-01,201701,SAT.,495.50,4,3.54,34.6
3,4,8,008,8 - PERRYSVILLE,Ross,Bus,2017-01-01,201701,SAT.,1480.00,4,3.54,34.6
4,5,11,011,11 - FINEVIEW,Ross,Bus,2017-01-01,201701,SAT.,208.00,4,3.54,34.6


### Add ZIP code to bus data

In [80]:
import geopandas as gpd

lat_lon = stopuse_df[['stop_id', 'routes_ser', 'longitude', 'latitude']].drop_duplicates().copy()
gs = gpd.points_from_xy(lat_lon['longitude'], lat_lon['latitude'])
gs

<GeometryArray>
[<POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>,
 <POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>, <POINT (-79.7 40.4)>,
 <POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>,
 <POINT (-79.8 40.4)>,
 ...
 <POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>,
 <POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>,
 <POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>, <POINT (-79.8 40.4)>,
 <POINT (-79.8 40.4)>]
Length: 6774, dtype: geometry

In [81]:
## Store long-lat data in geopandas dataframe
points = gpd.GeoDataFrame(lat_lon, geometry=gs, crs='epsg:4326')
points

,stop_id,routes_ser,longitude,latitude,geometry
0,E02110,"69, P69",-79.7600,40.3858,POINT (-79.76000 40.38580)
16,E02430,"69, P69",-79.7525,40.3864,POINT (-79.75250 40.38640)
32,E20030,"69, P69",-79.7576,40.3868,POINT (-79.75760 40.38680)
48,E20050,"69, P69",-79.7591,40.3857,POINT (-79.75910 40.38570)
64,E70818,"69, P69",-79.7503,40.3889,POINT (-79.75030 40.38890)
...,...,...,...,...,...
107551,E53290,69,-79.8077,40.3932,POINT (-79.80770 40.39320)
107563,E53300,69,-79.8093,40.3933,POINT (-79.80930 40.39330)
107575,E56995,69,-79.8098,40.3927,POINT (-79.80980 40.39270)
107587,E70960,69,-79.8098,40.3918,POINT (-79.80980 40.39180)


In [82]:
## Import Allegheny County ZIP polygons
zipcodes = gpd.read_file("Data/Allegheny_County_Zip_Code_Boundaries.shp")
zipcodes = zipcodes.to_crs(4326)
zipcodes.head()

,GDB_GEOMAT,OBJECTID,ZIP,NAME,ZIPTYPE,STATE,geometry
0,None,4,15007,BAKERSTOWN,NON-UNIQUE,PA,"POLYGON ((-79.92421 40.65180, -79.92421 40.651..."
1,None,5,15014,BRACKENRIDGE,NON-UNIQUE,PA,"POLYGON ((-79.73576 40.61128, -79.73574 40.611..."
2,None,6,15015,BRADFORD WOODS,NON-UNIQUE,PA,"POLYGON ((-80.07258 40.64491, -80.07240 40.644..."
3,None,8,15018,BUENA VISTA,NON-UNIQUE,PA,"POLYGON ((-79.78014 40.29273, -79.77960 40.292..."
4,None,14,15034,DRAVOSBURG,NON-UNIQUE,PA,"POLYGON ((-79.88627 40.35929, -79.88607 40.359..."


In [83]:
## Join the long-lat points and ZIP polygons
zip_points = points.sjoin(zipcodes, how='left')
zip_points[zip_points['ZIP'].isna()]

,stop_id,routes_ser,longitude,latitude,geometry,index_right,GDB_GEOMAT,OBJECTID,ZIP,NAME,ZIPTYPE,STATE
0,E02110,"69, P69",-79.7600,40.3858,POINT (-79.76000 40.38580),NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,E02430,"69, P69",-79.7525,40.3864,POINT (-79.75250 40.38640),NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,E20030,"69, P69",-79.7576,40.3868,POINT (-79.75760 40.38680),NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,E20050,"69, P69",-79.7591,40.3857,POINT (-79.75910 40.38570),NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,E70818,"69, P69",-79.7503,40.3889,POINT (-79.75030 40.38890),NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,E70819,"69, P69",-79.7484,40.3883,POINT (-79.74840 40.38830),NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,E70820,"69, P69",-79.7523,40.3865,POINT (-79.75230 40.38650),NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,E70821,"69, P69",-79.7568,40.3875,POINT (-79.75680 40.38750),NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,E70822,"69, P69",-79.7576,40.3870,POINT (-79.75760 40.38700),NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,E70823,"69, P69",-79.7587,40.3859,POINT (-79.75870 40.38590),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
## Account for missing ZIP data for ZIP codes just outside Allegheny County
conditions = [((zip_points['ZIP'].isna()) & (zip_points['routes_ser'] == '69, P69')), 
              ((zip_points['ZIP'].isna()) & (zip_points['routes_ser'] == '1')),
              ((zip_points['ZIP'].isna()) & (zip_points['routes_ser'] == '14'))]

values = ['15085', '15068', '15003']

zip_points['ZIP'] = np.select(conditions, values, default = zip_points['ZIP'])

zip_points[zip_points['ZIP'].isna()]

,stop_id,routes_ser,longitude,latitude,geometry,index_right,GDB_GEOMAT,OBJECTID,ZIP,NAME,ZIPTYPE,STATE


In [85]:
zip_points = zip_points[['stop_id', 'routes_ser', 'ZIP']].copy()

In [86]:
stopuse_df = stopuse_df.merge(zip_points, how = 'left', on = ['stop_id', 'routes_ser'])
stopuse_df

,clever_id,stop_id,stop_name,direction,routes_ser,latitude,longitude,mode,shelter,stop_type,...,route_name,serviceday,total_ons,total_offs,days,avg_ons,avg_offs,total_precip,avg_temp,ZIP
0,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.7600,Bus,No Shelter,Bus Stop,...,69,Sat,12.0,0.0,4,3.000000,0.000000,5.34,69.0,15085
1,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.7600,Bus,No Shelter,Bus Stop,...,69,Sun,14.0,0.0,6,2.333333,0.000000,5.34,69.0,15085
2,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.7600,Bus,No Shelter,Bus Stop,...,69,Weekday,64.0,1.0,20,3.200000,0.050000,5.34,69.0,15085
3,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.7600,Bus,No Shelter,Bus Stop,...,P69,Weekday,39.0,0.0,20,1.950000,0.000000,5.34,69.0,15085
4,7858,E02110,5TH ST AT CAVIT AVE,Inbound,"69, P69",40.3858,-79.7600,Bus,No Shelter,Bus Stop,...,69,Sat,11.0,0.0,4,2.750000,0.000000,3.69,35.2,15085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107606,22643,E70980,HERMAN AVE AT COMMERCE ST,Both,69,40.3925,-79.8086,Bus,No Shelter,Bus Stop,...,69,Sun,1.0,1.0,5,0.200000,0.200000,0.92,64.6,15148
107607,22643,E70980,HERMAN AVE AT COMMERCE ST,Both,69,40.3925,-79.8086,Bus,No Shelter,Bus Stop,...,69,Weekday,88.0,38.0,21,4.190476,1.809524,0.92,64.6,15148
107608,22643,E70980,HERMAN AVE AT COMMERCE ST,Both,69,40.3925,-79.8086,Bus,No Shelter,Bus Stop,...,69,Sat,1.0,1.0,4,0.250000,0.250000,2.67,51.3,15148
107609,22643,E70980,HERMAN AVE AT COMMERCE ST,Both,69,40.3925,-79.8086,Bus,No Shelter,Bus Stop,...,69,Sun,0.0,1.0,4,0.000000,0.250000,2.67,51.3,15148


## Output Data

In [88]:
stopuse_df.to_csv("Data/stop_usage.csv", index = False)
ridership_df.to_csv("Data/ridership.csv", index = False)

In [89]:
#demo_df.to_csv("Data/zip_demo.csv", index = False)
#commute_df.to_csv("Data/zip_commute.csv", index = False)